# Syväoppimisen harjoitustyö
Työn aiheena pistiäiskuvien luokittelu ja datan lähteenä https://www.kaggle.com/datasets/thedatasith/hymenoptera
Tarkoitus on siis erottaa Hymenoptera-lahkon hyönteisten kuvamateriaalista muurahaiset erilleen mesipistiäisistä. Taksonomisesti sekä muurahaiset että mesipistiäiset kuuluvat hoikkatyvisiin pistiäisiin, eli ovat varsin läheistä sukua toisilleen. Haastetta varmaankin riittää. Inspiraatiota aiheen valintaan tarjosi hyönteisharrastus ja kurssin päättymisviikonlopun BioBlitz Turku. Katsotaan mitä tästä tulee.

In [1]:
#tarvittavat kirjastot kootusti tähän
import tensorflow as tf
import glob
import random
import ipyplot # !pip install ipyplot
import matplotlib.pyplot as plt

In [2]:
#datan lataaminen ja esikäsittely. Data on ladattu paikalliseen hakemistoon.

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'data/Hymenoptera/train', labels='inferred', label_mode='int', class_names=None,
    color_mode='rgb', batch_size=32, image_size=(224, 224), shuffle=True,
    validation_split=None, subset=None, interpolation='bilinear'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'data/Hymenoptera/val', labels='inferred', label_mode='int', class_names=None,
    color_mode='rgb', batch_size=32, image_size=(224, 224), shuffle=True,
    validation_split=None, subset=None, interpolation='bilinear'
)
train_dataset

Found 242 files belonging to 2 classes.
Found 153 files belonging to 2 classes.


<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>

# oma kommentti
Kun aloin selvitellä, kuinka ulkoisen jpg-kuvan saa syötteeksi, löytyi tämmöinen kätevä työkalu. Tässä käytetty `image_dataset_from_directory()` osaa jakaa kuvat suoraan luokkiin, kunhan polun takana aukeaa oikeanlainen hakemistorakenne. Yllättävän kätevää. Vähemmän kätevää oli hahmottaa mitä tulokseksi saadaan.

Tässä `image_dataset_from_directory()` on `keras.preprosessing` alla, mutta Tensorflow 2.8 nähtävästi `keras.utils` alla.

In [3]:
#katsotaan luokkien nimet, että menee varmasti oikein
print(train_dataset.class_names)

['ants', 'bees']


In [3]:
#muutama satunnaisesti valittava esimerkkikuva datasta
file_path_1 = ['data/hymenoptera/train/ants/*.jpg']
file_path_2 = ['data/hymenoptera/train/bees/*.jpg']
ant_images = glob.glob(random.choice(file_path_1))
bee_images = glob.glob(random.choice(file_path_2))

#ylärivillä muurahaisia...
ipyplot.plot_images([random.choice(ant_images), random.choice(ant_images), random.choice(ant_images)],
                    img_width=180)
#...ja alarivillä mesipistiäisiä
ipyplot.plot_images([random.choice(bee_images), random.choice(bee_images), random.choice(bee_images)], 
                    img_width=180)

In [7]:
#Prefetch. Tarkoitus on nopeuttaa koneen toimintaa
train_data = train_dataset.prefetch(buffer_size=32)
test_data = test_dataset.prefetch(buffer_size=32)
train_data

<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>

# oma kommentti
Tämä on oikeastaan todella vaikea datasetti. Itse kohde on näissä useimmiten pieni osa kuvaa ja harvoin kovin puhtaasti edustettuna. Toisaalta näin se on luonnossakin. Myös ihan luontoharrastajan näkökulmasta pistiäisporukka on ärsyttävän valpasta ja otuksesta saa harvoin lähikuvaa. Kuvia voi tietysti jälkeenpäin croppailla, mutta muuten asian kanssa on elettävä, jos haluaa että työkalusta on käytännössä mitään hyötyä.

Esikäsittelyyn liittyvänä yksityiskohtana train-aineistossa oli mukana yksi verkkosiipinen ja kaksi muurahaista jäljittelevää hämähäkkiä. Kuvat kannatti silmäillä läpi.

Prefetch on käsittääkseni jonkinlaista välimuistittamista. Temppu löytyi, kun koetin löytää keinoja nopeuttaa analysointia.

In [5]:
#neuroverkon toteutus
#rakennetaan konvoluutioneuroverkko

input_layer = tf.keras.Input(shape=(224, 224, 3))
normalization = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1.0/255)(input_layer)
convolution1 = tf.keras.layers.Conv2D(48, kernel_size=(2, 2), padding='same', strides=1, activation='relu')(normalization)
pooling1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(convolution1)
convolution2 = tf.keras.layers.Conv2D(18, kernel_size=(2, 2), padding='same', strides=1, activation='relu')(pooling1)
pooling2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(convolution2)
dropout1 = tf.keras.layers.Dropout(0.2)(pooling2)
flattened = tf.keras.layers.Flatten()(dropout1)
dense1 = tf.keras.layers.Dense(8, activation='relu')(flattened)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dense1) #tulos on binäärisesti ant tai bee

tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

fit = model.fit(train_data, batch_size=10, epochs=10, validation_data=test_data)

Epoch 1/10
8/8 [==============================] - 16s 2s/step - loss: 0.9457 - accuracy: 0.4298 - val_loss: 0.7942 - val_accuracy: 0.4510
Epoch 2/10
8/8 [==============================] - 16s 2s/step - loss: 0.6696 - accuracy: 0.5909 - val_loss: 0.7116 - val_accuracy: 0.5556
Epoch 3/10
8/8 [==============================] - 16s 2s/step - loss: 0.6429 - accuracy: 0.6777 - val_loss: 0.6794 - val_accuracy: 0.5229
Epoch 4/10
8/8 [==============================] - 17s 2s/step - loss: 0.6165 - accuracy: 0.6901 - val_loss: 0.6899 - val_accuracy: 0.5359
Epoch 5/10
8/8 [==============================] - 19s 2s/step - loss: 0.5666 - accuracy: 0.6777 - val_loss: 0.6853 - val_accuracy: 0.5882
Epoch 6/10
8/8 [==============================] - 16s 2s/step - loss: 0.5423 - accuracy: 0.7479 - val_loss: 0.6655 - val_accuracy: 0.6078
Epoch 7/10
8/8 [==============================] - 15s 2s/step - loss: 0.4744 - accuracy: 0.8058 - val_loss: 0.7705 - val_accuracy: 0.5686
Epoch 8/10
8/8 [==================

# oma kommentti
Neuroverkko on rakennettu yksinkertaisesta input > conv2d > flattened > dense -rakenteesta alkaen, paremman teorian puutteessa pitkälti fiilispohjalta. Eräänlaisena esikäsittelynä `keras.layers.experimental.preprocessing.Rescaling()` skaalaa pikseliarvot. Kurssin tf versiossa 2.3 se on tuossa kirjastossa, uudemmissa suoraan `keras.layer` alla. Yksivärinen data olisi helpompaa ja nopeampaa käsitellä, mutta väri-informaatio on tässä aika olennaista. Useimmat muurahaislajit ovat yksivärisen tummanruskeita tai punaisia, kun mesipistäisissä myös keltaiset sävyt ja jonkinasteinen kirjavuus ovat hyvin tavallisia. 

Aloittaessa idea vaikutti hauskalta, mutta verkko näyttää pääsevän vain 50-60 % tarkkuuteen. Olen rehellisesti sanottuna tosi pettynyt tulokseen. Ei tälle nyt oikein mitään mielekästä jatkoakaan saa rakennettua. Vähän lässähti motivaatio nyt.

In [6]:
#tarkkuuden tarkastelu
loss, accuracy = model.evaluate(test_dataset, verbose=1)
print(f'Loss: {loss}, accuracy: {accuracy}')

5/5 [==============================] - 2s 329ms/step - loss: 0.6988 - accuracy: 0.6013
Loss: 0.6987718939781189, accuracy: 0.601307213306427


# oma kommentti
Työskentelyn iloa vähensi merkittävästi, se tosiasia, ettei läppärissä tunnu tehot riittävän kovin syviin malleihin. Jatkokehitystä varten pitää odotella että joku kustantaa paremmat resurssit.

Aineisto on kieltämättä vaikea. Kohde on usein pieni osa koko kuvasta ja sekä muurahaiset että mesipistiäiset muistuttavat toisiaan. Tämä ei nyt ollut oikeastaan kovin produktiivista. Työn päälimmäiseksi sisällöksi jäi oma, sinänsä parantunut, ymmärrys siitä, kuinka automaattinen lajintunnistus onnistuu... ja kuinka hankalaa se voikaan olla. Sovellusten käytössä kannattaa siis jatkossakin panostaa kuvan laatuun (kuten tietysti aina muutenkin).